In [2]:
# Init libraries
import warnings
import mne
import numpy as np
from sklearn.exceptions import ConvergenceWarning

np.random.seed(23)
mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )
warnings.filterwarnings(action = "ignore", category = ConvergenceWarning )

In [3]:
# Project defaults
# The root dir
root_dir = "./ds003626"

# Sampling rate
fs = 256

# Select the useful par of each trial. Time in seconds
t_start = 1.5
t_end = 3.5

In [4]:
# Load dataset
from aux.pre_process import get_subjects_data_and_label, get_subjects_data_label_group

condition = "Inner"
data, labels, groups = get_subjects_data_label_group(root_dir, condition, t_start = t_start, t_end = t_end, fs = fs)
len(data), len(labels), len(groups)

(10, 10, 10)

In [5]:
data_array=np.vstack(data)
label_array=np.hstack(labels)
group_array=np.hstack(groups)
data_array=np.moveaxis(data_array, 1, 2)

In [6]:
data_array.shape, label_array.shape, group_array.shape

((2236, 512, 128), (2236,), (2236,))

In [10]:
from others.eeg import EEGNet
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import rcParams
tf.random.set_seed(42)
from tensorflow.python.keras.layers import Conv1D, BatchNormalization, MaxPool1D, Dropout, AveragePooling1D, \
    GlobalAveragePooling1D, Dense, LeakyReLU
from tensorflow.python.keras import Sequential

def cnnmodel():
    _m=Sequential()
    _m.add(Conv1D(filters=5, kernel_size=3, strides=1, input_shape=(2236, 128)))
    _m.add(BatchNormalization())
    _m.add(LeakyReLU())
    _m.add(MaxPool1D(pool_size=2, strides=2))
    _m.add(Conv1D(filters=5, kernel_size=3, strides=1))
    _m.add(LeakyReLU())
    _m.add(MaxPool1D(pool_size=2, strides=2))
    _m.add(Dropout(0.5))
    _m.add(Conv1D(filters=5, kernel_size=3, strides=1))
    _m.add(LeakyReLU())
    _m.add(AveragePooling1D(pool_size=2, strides=2))
    _m.add(Dropout(0.5))
    _m.add(Conv1D(filters=5, kernel_size=3, strides=1))
    _m.add(LeakyReLU())
    _m.add(AveragePooling1D(pool_size=2, strides=2))
    _m.add(Conv1D(filters=5, kernel_size=3, strides=1))
    _m.add(LeakyReLU())
    _m.add(GlobalAveragePooling1D())
    _m.add(Dense(1, activation='sigmoid'))

    _m.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return _m

model = cnnmodel()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 2234, 5)           1925      
_________________________________________________________________
batch_normalization_1 (Batch (None, 2234, 5)           20        
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 2234, 5)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1117, 5)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1115, 5)           80        
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 1115, 5)           0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 557, 5)           

In [1]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gfk=GroupKFold()
accuracy=[]
for train_index, val_index in gfk.split(data_array, label_array, groups=group_array):
    train_features, train_labels = data_array[train_index], label_array[train_index]
    val_features, val_labels = data_array[val_index], label_array[val_index]
    scaler=StandardScaler()
    train_features=scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features=scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features, train_labels, epochs=10, batch_size=100, validation_data=(val_features, val_labels))
    accuracy.append(model.evaluate(val_features, val_labels)[1])

    break

NameError: name 'data_array' is not defined